In [96]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [97]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

directory = '../input/5-celebrity-faces-dataset/train/'

Name=[]
for file in os.listdir(directory):
    Name+=[file]
print(Name)
print(len(Name))

N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

dataset=[]
# testset=[]
count=0
for file in os.listdir(directory):
    path=os.path.join(directory,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(224,224))
        image=img_to_array(image)
        image=image/255.0
        t+=1
        dataset.append([image,count])
    count+=1

data,labels0=zip(*dataset)
labels1=(labels0)
data=np.array(data)
labels=np.array(labels1)

print(data.shape)
print(labels.shape)

In [98]:

valdir = "../input/5-celebrity-faces-dataset/val/"

valname=[]

for file in os.listdir(valdir):
    valname+=[file]
print(valname)
print(len(valname))

N1=[]
for i in range(len(valname)):
    N1+=[i]
    
normal_mapping=dict(zip(valname,N1)) 
reverse_mapping=dict(zip(N1,valname))  


valset=[]
count1=0
for file in os.listdir(valdir):
    path=os.path.join(valdir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(224,224))
        image=img_to_array(image)
        image=image/255.0
        t+=1
        valset.append([image,count1])
    count1+=1
    
    
    
data1,labels12=zip(*valset)
labels1=(labels12)
valx=np.array(data1)
valy=np.array(labels1)

# tlabels1=to_categorical(tlabels0)
# test=np.array(test)
# tlabels=np.array(tlabels1)

# trainx00,test00,trainy00,testy00=train_test_split(data,labels,test_size=0.9,random_state=44)
# trainx0,testx,trainy0,testy=train_test_split(trainx00,trainy00,test_size=0.2,random_state=44)
# trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

# print(trainx.shape)
# print(testx.shape)
# print(trainy.shape)
# print(testy.shape)

print(valx.shape)
print(valy.shape)

# datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
#                         width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")



In [99]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses

model = models.Sequential()
model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=data.shape[1:]))
model.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

In [101]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
# labels.shape
history = model.fit(data, labels, batch_size=5, epochs=20, validation_data=(valx, valy))

In [102]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

In [106]:
model.evaluate(valx, valy)